# 파이썬으로 메일 보내기

## 메일 서버 접속해서 로그인 후 메일 보내기

### 기본 개념

메일을 보내는 코드를 작성하기 위해서 알아야 할 최소한의 개념부터 살펴보자. 우선 우리가 메일을 보낼 때 서버에서 실제로 일어나는 일을 이해해야 한다.

#### SMTP
https://ko.wikipedia.org/wiki/%EA%B0%84%EC%9D%B4_%EC%9A%B0%ED%8E%B8_%EC%A0%84%EC%86%A1_%ED%94%84%EB%A1%9C%ED%86%A0%EC%BD%9C

우리가 메일을 보내는 행위는 결국 메일 서비스를 제공하는 회사에게 "이런 주소로 이런 내용을 담아 발송해줘"라는 요청을 하는 것이고, 이 회사는 자신의 서버를 통해 발신자의 요청사항을 처리해주게 되는데, 이 때 요청을 처리하는 서버를 SMTP 서버라고 부른다. (SMTP는 Simple Mail Transfer Protocol의 약자로, 이메일을 주고 받는 표준적인 약속 체계나 규약을 의미한다. 즉, 우리가 SMTP 서버에게 미리 약속된 형식으로 요청을 해야 서버에서 그 요청을 처리한다는 뜻이다.)

서버에게 메일 발송 요청을 하려면 서버의 주소와 메일 발송에 사용할 포트(경로)만 알면 된다.

대표적인 메일 서비스들의 주소와 포트를 나열하면 다음과 같다.
- 구글: smtp.gmail.com (465)
  - https://myaccount.google.com/u/4/lesssecureapps (보안 수준이 낮은 앱 허용: 사용)
- 네이버: smtp.naver.com (465)
  - https://help.naver.com/support/contents/contents.help?serviceNo=2342&categoryNo=2281
- 다음: smtp.daum.net (465)
  - https://cs.daum.net/faq/266/12146.html?page=1#18007

465이 아닌 587 포트를 사용하기도 하는데, 본 강의에서는 465로 실습을 진행한다. (465는 SSL, 587은 TLS라는 암호화 프로토콜을 사용하는데, 이에 대한 설명은 생략한다.)

만약 회사 메일서버를 사용하고 싶다면 마찬가지로 SMTP 서버 주소와 포트를 알아내어 적용하면 된다.

(참고로 SMTP 설정 과정에서 IMAP/POP3라는 단어도 많이 마주하게 되는데, SMTP가 메일 발송을 위한 프로토콜이었다면, IMAP/POP3는 메일 수신을 위한 프로토콜이라 이해하면 된다.)

<img src="https://media.vlpt.us/images/snowman39/post/4d3dbf46-0e2b-4242-8f8d-aa72c17f3f74/%E1%84%89%E1%85%B3%E1%84%8F%E1%85%B3%E1%84%85%E1%85%B5%E1%86%AB%E1%84%89%E1%85%A3%E1%86%BA%202020-08-26%20%E1%84%8B%E1%85%A9%E1%84%92%E1%85%AE%205.05.53.png">

#### MIME
- https://ko.wikipedia.org/wiki/MIME
- https://developer.mozilla.org/ko/docs/Web/HTTP/Basics_of_HTTP/MIME_types

MIME(Multipurpose Internet Mail Extensions)은 메일 작성을 위한 표준적인 형식을 의미한다. 우리가 메일 서버에게 "이런 내용으로 메일을 작성해서 보내줘"라고 요청할 때는 서버가 이해할 수 있는 MIME 형식으로 메일을 작성해서 요청해야 한다는 뜻이다.

결국 요약하자면 메일 발송을 위해 준비할 것은 SMTP 서버 주소와 포트, 이메일 계정과 비밀번호 뿐이다. 이후에 파이썬 코드를 통해 메일과 관련한 정보들을 MIME이라는 템플릿에 담아 서버에게 발송을 요청하게 된다. (파이썬 기본으로 내장된 라이브러리만으로 구현이 가능하기 때문에 별도의 라이브러리는 설치하지 않는다.)

### 코드 예제

일단 코드를 실행해서 메일이 정상적으로 발송되는지 확인하자.

In [1]:
from account import *
from email.message import EmailMessage
from smtplib import SMTP_SSL

# 템플릿 생성
msg = EmailMessage()

# 보내는 사람 / 받는 사람 / 제목 입력
msg["From"] = MY_ID
msg["To"] = "hleecaster@gmail.com"
msg["Subject"] = "제목입니다."

# 본문 구성
본문 = """안녕하세요.

본문입니다.

감사합니다."""

msg.set_content(본문)

with SMTP_SSL("smtp.gmail.com", 465) as smtp:
    smtp.login(MY_ID, MY_PW)
    smtp.send_message(msg)

그렇다면 이번에는 이메일을 보내는 함수를 만들어서 사용해보자.

In [2]:
from account import *
from email.message import EmailMessage
from smtplib import SMTP_SSL

def send_mail(받는사람, 제목, 본문):

    # 템플릿 생성
    msg = EmailMessage()

    # 보내는 사람 / 받는 사람 / 제목 입력
    msg["From"] = MY_ID
    msg["To"] = "hleecaster@gmail.com"
    msg["Subject"] = 제목

    # 본문 구성
    msg.set_content(본문)

    with SMTP_SSL("smtp.gmail.com", 465) as smtp:
        smtp.login(MY_ID, MY_PW)
        smtp.send_message(msg)

In [3]:
send_mail("hleecaster@gmail.com", "제목2입니다", "본문2입니다")

## 메일 본문 HTML 형식으로 보내기

앞선 예제는 이메일 본문에 서식이 없는 plain 텍스트 형식이다.

그러나 만약 웹페이지처럼 화려한 이메일 본문을 작성하고 싶다면, 그리고 HTML이라는 웹페이지의 골격을 구성하는 언어를 이해하고 있다면 이메일 본문을 HTML 작성해서 보내는 것도 가능하다.

### 코드 예제

<code>msg.set_content(본문)</code>에서 괄호 안에 <code>subtype='html'</code>을 명시한다.

In [4]:
from account import *
from email.message import EmailMessage
from smtplib import SMTP_SSL

def send_mail(받는사람, 제목, 본문):

    # 템플릿 생성
    msg = EmailMessage()

    # 보내는 사람 / 받는 사람 / 제목 입력
    msg["From"] = MY_ID
    msg["To"] = "hleecaster@gmail.com"
    msg["Subject"] = 제목

    # 본문 구성
    msg.set_content(본문, subtype='html')

    with SMTP_SSL("smtp.gmail.com", 465) as smtp:
        smtp.login(MY_ID, MY_PW)
        smtp.send_message(msg)

In [5]:
본문 = """
<html>
    <head></head>
    <body>
        <h2>안녕하세요!</h2>
        <h3>HTML 테스트 메일입니다.</h3>
        <img src="https://www.google.com/images/branding/googlelogo/1x/googlelogo_color_272x92dp.png">
        <p><a href="http://hleecaster.com/">블로그 바로가기</a></p>
    </body>
</html>
"""
send_mail("hleecaster@gmail.com", "제목입니다", 본문)

## 메일에 파일 첨부하여 보내기

기존 함수에 파일을 첨부하는 코드를 추가해보자.

### 코드 예제

In [6]:
from account import *
from email.message import EmailMessage
from smtplib import SMTP_SSL
from pathlib import Path

def send_mail(받는사람, 제목, 본문, 첨부파일=False):

    # 템플릿 생성
    msg = EmailMessage()

    # 보내는 사람 / 받는 사람 / 제목 입력
    msg["From"] = MY_ID
    msg["To"] = 받는사람
    msg["Subject"] = 제목

    # 본문 구성
    msg.set_content(본문)
    
    # 파일 첨부
    if 첨부파일:
        파일명 = Path(첨부파일).name
        with open(첨부파일, "rb") as f:
            msg.add_attachment(f.read(), maintype="application", subtype="octet-stream", filename=파일명)
            msg.add_header('Content-Disposition', 'attachment', filename=파일명)

    with SMTP_SSL("smtp.gmail.com", 465) as smtp:
        smtp.login(MY_ID, MY_PW)
        smtp.send_message(msg)

In [7]:
send_mail("hleecaster@gmail.com", "제목입니다", "본문입니다", "files/첨부1.pdf")

## (실습) 제목, 본문, 첨부파일 다르게 대량으로 이메일 보내기

In [8]:
from account import *
from email.message import EmailMessage
from smtplib import SMTP_SSL
from pathlib import Path
from openpyxl import load_workbook


def send_mail(받는사람, 제목, 본문, 첨부파일=False):

    # 템플릿 생성
    msg = EmailMessage()

    # 보내는 사람 / 받는 사람 / 제목 입력
    msg["From"] = MY_ID
    msg["To"] = 받는사람
    msg["Subject"] = 제목

    # 본문 구성
    msg.set_content(본문)
    
    # 파일 첨부
    if 첨부파일:
        파일명 = Path(첨부파일).name
        with open(첨부파일, "rb") as f:
            msg.add_attachment(f.read(), maintype="application", subtype="octet-stream", filename=파일명)
            msg.add_header('Content-Disposition', 'attachment', filename=파일명)
            
    with SMTP_SSL("smtp.gmail.com", 465) as smtp:
        smtp.login(MY_ID, MY_PW)
        smtp.send_message(msg)
    
    # 완료 메시지
    print(받는사람, "성공", sep="\t")

        
# 엑셀파일에서 가져온 정보를 활용해 함수 반복 실행
wb = load_workbook('files/이메일리스트.xlsx', data_only=True)
ws = wb["명단"]

for 행 in ws.iter_rows():
    받는사람 = 행[0].value
    제목 = 행[1].value
    본문 = 행[2].value
    첨부파일 = 행[3].value
    send_mail(받는사람, 제목, 본문, 첨부파일)

hleecaster@gmail.com	성공
hleecaster@gmail.com	성공
hleecaster@gmail.com	성공
hleecaster@gmail.com	성공
